# Cleaning data

Each semester, we apply a multiple choice electromagnetism test on the first and last week of the Basics Electromagnetism course. Since 2020, the test is done on a virtual plataform and a file with the students' answers is exported from the plataform. We call the test from the first week pre-test or T1 and the test from the last week post-test or T2

In [1]:
import pandas as pd
import numpy as np
import unicodedata

In [2]:
# importing dictionary
answer_dict = pd.read_json('answer_dict.json', typ='series')

In [3]:
# pre-test and post-test data
data_t1 = pd.read_csv('./2022_1/2022_1_T1.csv')
data_t2 = pd.read_csv('./2022_1/2022_1_T2.csv')

In [4]:
# replacing the "-" text from non answered questions for NaN
data_t1.replace('-',np.nan, inplace=True)
data_t2.replace('-',np.nan, inplace=True)

The datasets datat1 and datat2 are the answers of the pre-test and post-test, respectively. They contain the following columns:

| Column name | Column description | 
| --- | --- |
| Sobrenome | Student's last name |
| Nome | Student's first name |
| Endereço de email | Student's email adress |
| Estado | If the test was completed |
| Iniciado em | Date and time of when the student started the test |
| Completo | Date and time of when the student finished the test |
| Tempo utiliado | Raw text of how long the student took to complet the test |
| Avaliar/10,00 | Student's grade in the test |
| Resposta 1-31 | Students's selected answer in the question |

In [5]:
# final grades from students in each class/professor
a1 = pd.read_csv('./2022_1/2022_1_A1_PEDRO_LANA.csv')
a2 = pd.read_csv('./2022_1/2022_1_A2_GABRIELA.csv')
b1 = pd.read_csv('./2022_1/2022_1_B1_CARLOS_HEITOR.csv')
b2 = pd.read_csv('./2022_1/2022_1_B2_VITOR.csv')
m1 = pd.read_csv('./2022_1/2022_1_M1_CARLOS_HEITOR.csv')
m2 = pd.read_csv('./2022_1/2022_1_M2_LEO_CAMPOS.csv')
m3 = pd.read_csv('./2022_1/2022_1_M3_FILIPE.csv')
n1 = pd.read_csv('./2022_1/2022_1_N1_UBIRAJARA.csv')
n2 = pd.read_csv('./2022_1/2022_1_N2_LEO_CAMPOS.csv')
e1 = pd.read_csv('./2022_1/2022_1_E1_LUIZ_GUSTAVO.csv')
f1 = pd.read_csv('./2022_1/2022_1_F1_LUIZ_GUSTAVO.csv')

grades = pd.concat([a1,a2,b1,b2,m1,m2,m3,n1,n2,e1,f1])

The datasets a-f 1-3 contain following columns:

| Column name | Column description | 
| --- | --- |
| Aluno | Student's name |
| Semestre |Semester |
| Turma | Student's class |
| Professor | Student's professor |
| Metodo | Teaching method used by the professor |
| Nota final | Student's final grade |

The datasets are merged in the dataset grades, containing these informations about all the students of this semester

In [6]:
# deleting columns
data_t1.drop( columns = ['Endereço de email', 'Estado', 'Iniciado em', 'Completo', 'Avaliar/10,00'], inplace=True  )
data_t2.drop( columns = ['Endereço de email', 'Estado', 'Iniciado em', 'Completo', 'Avaliar/1,00'], inplace=True  )

In [7]:
# merging name and last name in a single column and deleting columns of name and last name
data_t1.insert( 0, 'student', data_t1["Nome"] + ' ' + data_t1["Sobrenome"] )
data_t1.drop( columns = ["Nome", "Sobrenome"], inplace=True )

data_t2.insert( 0, 'student', data_t2["Nome"] + ' ' + data_t2["Sobrenome"] )
data_t2.drop( columns = ["Nome", "Sobrenome"], inplace=True )

In [8]:
# renaming columns
data_t1.rename(columns = {'Tempo utilizado': 'T1_time'}, inplace = True)
data_t2.rename(columns = {'Tempo utilizado': 'T2_time'}, inplace = True)
grades.rename(columns = {'Aluno': 'student', 'Semestre':'semester', 'Turma':'class', 'Professor':'professor', 
                          'Metodo':'method', 'Nota Final':'final_grade'}, inplace = True)

In [9]:
# remove diacritics and double or non-standard whitespaces from students' and professors' names, put names in all upper case
data_t1['student'] = data_t1['student'].str.normalize('NFKD')\
                            .str.encode('ascii', errors='ignore')\
                            .str.decode('utf-8')
data_t1.student = data_t1.student.replace('\s+', ' ', regex=True)
data_t1['student'] = data_t1['student'].str.upper()

data_t2['student'] = data_t2['student'].str.normalize('NFKD')\
                            .str.encode('ascii', errors='ignore')\
                            .str.decode('utf-8')
data_t2.student = data_t2.student.replace('\s+', ' ', regex=True)
data_t2['student'] = data_t2['student'].str.upper()

grades['student'] = grades['student'].str.normalize('NFKD')\
                            .str.encode('ascii', errors='ignore')\
                            .str.decode('utf-8')
grades.student = grades.student.replace('\s+', ' ', regex=True)
grades['student'] = grades['student'].str.upper()

grades['professor'] = grades['professor'].str.normalize('NFKD')\
                            .str.encode('ascii', errors='ignore')\
                            .str.decode('utf-8')
grades.professor = grades.professor.replace('\s+', ' ', regex=True)
grades['professor'] = grades['professor'].str.upper()

In [10]:
# transforming time from raw text, e.g. "32 minutos e 20 segundos", to a numeric of how many seconds the student took to complete the test
T1_minutes = pd.to_numeric( data_t1['T1_time'].str.extract( '([0-9]+) minutos?', expand=False) )
T1_seconds = pd.to_numeric( data_t1['T1_time'].str.extract( '([0-9]+) segundos?', expand=False) )


# fix NaN in entries of the form 10 seconds (with implicit 0 minutes)
T1_minutes[ np.logical_and( 
    T1_seconds.notnull(),
    T1_minutes.isnull()
) ] = 0.0

# fix NaN in entries of the form 45 minutes (with implicit 0 seconds)
T1_seconds[ np.logical_and( 
    T1_seconds.isnull(),
    T1_minutes.notnull()
) ] = 0.0

data_t1['T1_time'] = 60*T1_minutes + T1_seconds

In [11]:
T2_minutes = pd.to_numeric( data_t2['T2_time'].str.extract( '([0-9]+) minutos?', expand=False) )
T2_seconds = pd.to_numeric( data_t2['T2_time'].str.extract( '([0-9]+) segundos?', expand=False) )

# fix NaN in entries of the form 10 seconds (with implicit 0 minutes)
T2_minutes[ np.logical_and( 
    T2_seconds.notnull(),
    T2_minutes.isnull()
) ] = 0.0

# fix NaN in entries of the form 45 minutes (with implicit 0 seconds)
T2_seconds[ np.logical_and( 
    T2_seconds.isnull(),
    T2_minutes.notnull()
) ] = 0.0

data_t2['T2_time'] = 60*T2_minutes + T2_seconds

In [12]:
# replacing plataform-outputed answers for the answer letter
data_t1.replace( answer_dict, inplace=True )
data_t2.replace( answer_dict, inplace=True )

data_t1.rename( columns = { 'Resposta '+str(i): 'T1Q'+str(i) for i in range(1,31+1) }, inplace=True )
data_t2.rename( columns = { 'Resposta '+str(i): 'T2Q'+str(i) for i in range(1,31+1) }, inplace=True )

In [13]:
# merge dataframes
data_tests = pd.merge(data_t1, data_t2, how='outer', on='student')
data = pd.merge(data_tests, grades, how='outer', on='student')

In [14]:
data.to_csv("data_2022_1.csv", index=False)